# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [7]:
df.describe()

,itemInSession,length,sessionId,userId
count,6820.000000,6820.000000,6820.000000,6820.000000
mean,22.761144,247.032221,599.181818,54.681232
std,23.444636,102.975921,284.953333,28.162734
min,0.000000,15.855870,3.000000,2.000000
25%,4.000000,197.321998,374.000000,29.000000
50%,15.000000,232.972605,605.000000,49.000000
75%,35.000000,274.121992,834.000000,80.000000
max,126.000000,2594.873020,1114.000000,101.000000


In [8]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

# Part II. Complete the Apache Cassandra coding portion of your project. 

## CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# Create a new keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [ ]:
# Set the keyspace for the session
session.set_keyspace('music_app')

### Query 1: get the artist, song title and song length for sessionId = 338 and itemInSession = 4

For this query, session_id and item_in_session are used as the composite primary key for the song_info_by_session table.

In [13]:
# Create the table for query 1
session.execute("""
    CREATE TABLE IF NOT EXISTS song_info_by_session (       
        session_id int,
        item_in_session int,
        artist text,
        song text,
        length decimal,        
        PRIMARY KEY (session_id, item_in_session)
    )
""")

# Insert data into the table
session.execute("""
    INSERT INTO song_info_by_session (        
        session_id,
        item_in_session,
        artist,
        song,
        length)
    VALUES (338, 4, 'Faithless', 'Music Matters (Mark Knight Dub)', 495.3073)
""")

# Execute the query to get the artist, song title and song length for sessionId = 338 and itemInSession = 4
result = session.execute("""
    SELECT artist, song, length
    FROM song_info_by_session
    WHERE session_id = 338 AND item_in_session = 4
""")

# Print the result
for row in result:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [14]:
# Insert data from the CSV file into the table
file = 'event_datafile_new.csv'

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute("""
            INSERT INTO song_info_by_session (session_id, item_in_session, artist, song, length)
            VALUES (%s, %s, %s, %s, %s)
        """, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
# Select all the artisrs, songs and song's length from table 'song_info_by_session' where session_id = 338 AND item_in_session = 4
result = session.execute("""
    SELECT artist, song, length
    FROM song_info_by_session
    WHERE session_id = 338 AND item_in_session = 4
""")

# Print the result
for row in result:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Queries 2: get all name of artist, song (sorted by itemInSession) and user (first and last name)  for userid = 10, sessionid = 182

In this query, the partition key is composed of user_id and session_id, and item_in_session is the clustering column.

In [16]:
# first creat song_info_by_userid table
session.execute("""
    CREATE TABLE IF NOT EXISTS song_info_by_userid (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song text,       
        first_name text,
        last_name text,       
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")

# Insert data from the CSV file into the table
file = 'event_datafile_new.csv'
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute("""
            INSERT INTO song_info_by_userid (user_id, session_id, item_in_session, artist, song, first_name, last_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Select all the artisrs, songs and song's length from table 'song_info_by_session' where for user_id = 10, session_id = 182, sorted by item_in_session
result = session.execute("""
    SELECT artist, song, first_name, last_name
    FROM song_info_by_userid
    WHERE user_id = 10 AND session_id = 182
    ORDER BY item_in_session
""")

# Print the result
for row in result:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Get all user name (first and last) in music app history who listened to the song 'All Hands Against His Own'

In this query, the partition key is song and user_id is the clustering column.

In [19]:
# creat 3rd table song_info_by_name
session.execute("""
    CREATE TABLE IF NOT EXISTS song_info_by_name (
        song text,
        user_id int,    
        first_name text,
        last_name text,      
        PRIMARY KEY ((song), user_id)
    )
""")

# Insert data from the CSV file into the table
file = 'event_datafile_new.csv'
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute("""
            INSERT INTO song_info_by_name (song, user_id, first_name, last_name)
            VALUES (%s, %s, %s, %s)
        """, ( line[9], int(line[10]), line[1], line[4]))

# Select every user name (first and last) in song_info_by_name table who listened to the song 'All Hands Against His Own'
result = session.execute("""
    SELECT first_name, last_name
    FROM song_info_by_name
    WHERE song = 'All Hands Against His Own'
""")

# Print the result
for row in result:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
# drop table song_info_by_session
session.execute("DROP TABLE IF EXISTS song_info_by_session")

In [21]:
# drop table song_info_by_userid
session.execute("DROP TABLE IF EXISTS song_info_by_userid")

In [22]:
# drop table song_info_by_name
session.execute("DROP TABLE IF EXISTS song_info_by_name")

### Close the session and cluster connection

In [23]:
session.shutdown()
cluster.shutdown()